# Modelo de predicción: Temperatura ambiente
En este notebook se entrenarán diferentes algoritmos de ML para desarrollar el modelo de predicción de la temperatura ambiente horaria con los datasets obtenidos de:


**Radiación del día anterior a la llamada** https://opendata.aemet.es/centrodedescargas/productosAEMET<br>
**Radiación solar de dos días antes del día de obtención de datos** http://www.soda-pro.com/web-services/radiation/cams-radiation-service<br>
**Datos climáticos de los cinco días anteriores a la llamada** https://openweathermap.org/api/one-call-api#history<br>
**Predicción climática de los dos días siguientes a la llamada** https://openweathermap.org/api/one-call-api<br>

- [Preparación](#Preparación)<br>

### 1. [Se importan los datos a usar para entrenar y validar el modelo](#Se-importan-los-datos-a-usar-para-entrenar-y-validar-el-modelo)

### 2. [Se dividen los datos en train y test](#Se-dividen-los-datos-en-train-y-test)

### 3. [Modelo base](#Modelo-base)

### 4. [Regresión lineal](#Regresión-lineal)

### 5. [k-Nearest Neighbors](#k-Nearest-Neighbors)

### 6. [Árbol de decisión](#Árbol-de-decisión)

### 7. [Regresión polinomial](#Regresión-polinomial)

### 8. [Gradient boosting](#Gradient-boosting)

### 9. [Random forest](#Random-forest)

### 10. [GridSearchCV](#GridSearchCV)

### 11. [Regresión lineal con GridSearchCV](#Regresión-lineal-con-GridSearchCV)

### 12. [k-Nearest Neighbors con GridSearchCV](#k-Nearest-Neighbors-con-GridSearchCV)

### 13. [Árbol de decisión con GridSearchCV](#Árbol-de-decisión-con-GridSearchCV)

### 14. [Gradient boosting con GridSearchCV](#Gradient-boosting-con-GridSearchCV)

### 15. [Random forest con Regressor RandomizedSearchCV](#Random-forest-con-Regressor-RandomizedSearchCV)

### 16. [Tensorflow](#Tensorflow)

### 17. [Selección de modelo](#Selección-de-modelo)

### 18. [Se comprueba que no haya sobreajuste](#Se-comprueba-que-no-haya-sobreajuste)

### 19. [Se guarda el modelo](#Se-guarda-el-modelo)


## Nota previa
Para poder ejecutar el script adecuadamente hay que utilizar la siguiente línea de comando para abrir el notebook desde la terminal:

In [1]:
#jupyter-lab --NotebookApp.iopub_data_rate_limit=1.0e15

**PRECAUCIÓN:** Tarda muchas horas en correr

## Preparación

Se fija el directorio de trabajo

In [2]:
%cd /home/dsc/git/TFM/

/home/dsc/git/TFM


In [3]:
directorio = '/home/dsc/git/TFM/'

### ``dividir_train_test()``
Esta función divide los datos en sets de train y de test:
- Train: prop (Por defecto, 80%)
- Test: 100% - prop

In [4]:
def dividir_train_test(x,y, prop = 0.8):
    
    # Proporción de train
    tam_train = prop

    # Divido en train y test
    x_train, x_test, y_train, y_test = train_test_split(x, y, train_size = tam_train, random_state = 1)

    print('x_train: {}%. Nº de datos: {}'.format((len(x_train)/len(x))*100, len(x_train)))
    print('y_train: {}%. Nº de datos: {}'.format((len(y_train)/len(y))*100, len(y_train)))


    print('x_test: {}%. Nº de datos: {}'.format((len(x_test)/len(x))*100, len(x_test)))
    print('y_test: {}%. Nº de datos: {}'.format((len(y_test)/len(y))*100, len(y_test)))
    
    return x_train, x_test, y_train, y_test

### ``graf_compara()``

Esta función representa mediante barras verticales los valores reales frente a los predichos.  

In [5]:
def graf_compara(nombre_modelo, y_real, y_pred):
    
    # Valores predicción
    predic = pd.DataFrame({'Dato': y_pred})
    predic.insert(len(predic.columns),"index",[i for i in range(0,len(predic["Dato"]))],True)
    
    # Valores reales
    real = pd.DataFrame({'Dato': y_real})
    real.insert(len(real.columns),"index",[i for i in range(0,len(real["Dato"]))],True)
    
    # Comparación
    comparacion = pd.concat([real, predic], keys=["Real", "Prediccion"]).reset_index()
    comparacion.drop(['level_1'], axis=1, inplace = True)
    comparacion.columns = ['Tipo', 'Dato', "Index"]
    #print(comparacion)

    sns.catplot(data = comparacion, kind = "bar", x = "Index", y = "Dato", hue = "Tipo", estimator = np.median, height = 10, aspect = 5)

### ``mape_fun()``

In [6]:
def mape_fun(y_real, y_pred): 
    y_real, y_pred = np.array(y_real), np.array(y_pred)
    return np.mean(np.abs((y_real - y_pred) / y_real)) * 100

### ``metricas()``

Esta función calcula las diferentes métricas de la predicción arrojada por el modelo  

Métricas: ``mae``, ``mse``,``rmse``, ``r2``, ``mape``

In [7]:
def metricas(modelo, y_real, y_pred):
    
    from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
    
    # MAE: el error se calcula como un promedio de diferencias absolutas entre los valores objetivo y las predicciones. Todas las diferencias individuales se ponderan por igual en el promedio.
    mae = mean_absolute_error(y_real, y_pred)

    # MSE: mide el error cuadrado promedio de las predicciones. Para cada punto, calcula la diferencia cuadrada entre las predicciones y el objetivo y luego promedia esos valores.
    mse = mean_squared_error(y_real, y_pred, squared = False)
    
    # RMSE: es la raíz cuadrada de MSE. Tiene la escala de la variable objetivo.
    rmse = np.sqrt(mean_squared_error(y_real, y_pred))
    
    # R^2: está estrechamente relacionada con la MSE, pero tiene la ventaja de estar libre de escala. Está siempre entre -∞ y 1.
    r2 = r2_score(y_real, y_pred)
    
    # MAPE: Para cada objeto, el error absoluto se divide por el valor objetivo, dando un error relativo.
    #mape = mape_fun(y_real, y_pred)
    
    
    print('MODEL: ', modelo)
    print('MAE: ', mae)
    print('MSE: ', mse)
    print('RMSE: ', rmse)
    print('R2 : ', r2)
    #print('MAPE : ', mape)
    
    #return modelo, mae, mse, rmse, r2, mape
    return modelo, mae, mse, rmse, r2

### ``compracion_metricas()``

Esta función representa las métricas de cada modelo entrenado, para poder compararlas.

In [8]:
def compracion_metricas(lista_modelos):
    
    plt.style.use('ggplot')
    
    # Creamos dataframes para albergar las métricas
    df_mae = pd.DataFrame(columns = ['mae', "modelo"])
    df_mse = pd.DataFrame(columns = ['mse', "modelo"])
    df_rmse = pd.DataFrame(columns = ['rmse', "modelo"])
    df_r2 = pd.DataFrame(columns = ['r2', "modelo"])
    #df_mape = pd.DataFrame(columns = ['mape', "modelo"])

    # Se llenan los dataframes con las métricas
    for modelo in lista_modelos:
        
        df_mae = df_mae.append({'mae': modelo[1], "modelo": modelo[0]}, ignore_index=True)
        df_mse = df_mse.append({'mse': modelo[2], "modelo": modelo[0]}, ignore_index=True)
        df_rmse = df_rmse.append({'rmse': modelo[3], "modelo": modelo[0]}, ignore_index=True)
        df_r2 = df_r2.append({'r2': modelo[4], "modelo": modelo[0]}, ignore_index=True)
        #df_mape = df_mape.append({'mape': modelo[5], "modelo": modelo[0]}, ignore_index=True)
    
    # Se crea la figura y añado los subplots de cada métrica
    fig = plt.figure(figsize = (15, len(lista_modelos*4)))
    ax1 = fig.add_subplot(5,1,1)
    ax2 = fig.add_subplot(5,1,2)
    ax3 = fig.add_subplot(5,1,3)
    ax4 = fig.add_subplot(5,1,4)
    #ax5 = fig.add_subplot(5,1,5)
    
    #MAE
    fig = sns.barplot(x = "mae", y = "modelo", data = df_mae, ax = ax1, orient = "h", color = 'green').set_title("Comparación de métricas")
    ax1.tick_params(labelbottom = False, bottom = False)
    ax1.set_xlabel("MAE")
    ax1.set_ylabel(" ")
    for pa in ax1.patches:
        ax1.annotate("%.4f" % pa.get_width(), xy = (pa.get_width(), pa.get_y() + pa.get_height()/2),
            xytext = (5, 0), textcoords = 'offset points', ha = "left", va = "center")
    
    #MSE
    fig = sns.barplot(x = "mse", y = "modelo", data = df_mse, ax = ax2, orient = "h", color = 'red')
    ax2.tick_params(labelbottom = False, bottom = False)
    ax2.set_xlabel("MSE")
    ax2.set_ylabel(" ")
    for pa in ax2.patches:
        ax2.annotate("%.4f" % pa.get_width(), xy = (pa.get_width(), pa.get_y() + pa.get_height()/2),
            xytext = (5, 0), textcoords = 'offset points', ha = "left", va = "center")
        
    #RMSE
    fig = sns.barplot(x = "rmse", y = "modelo", data = df_rmse, ax = ax3, orient = "h", color = 'blue')
    ax3.tick_params(labelbottom = False, bottom = False)
    ax3.set_xlabel("RMSE")
    ax3.set_ylabel(" ")
    for pa in ax3.patches:
        ax3.annotate("%.4f" % pa.get_width(), xy = (pa.get_width(), pa.get_y() + pa.get_height()/2),
            xytext = (5, 0), textcoords = 'offset points', ha = "left", va = "center")

    #R2
    fig = sns.barplot(x = "r2", y = "modelo", data = df_r2, ax = ax4, orient = "h", color = 'yellow')
    ax4.set_xlabel("R2")
    ax4.set_ylabel(" ")
    ax4.tick_params(labelbottom = False, bottom = False)
    for pa in ax4.patches:
        ax4.annotate("%.4f" % pa.get_width(), xy = (pa.get_width(), pa.get_y() + pa.get_height()/2),
            xytext = (5, 0), textcoords = 'offset points', ha = "left", va = "center")
        
    #MAPE
    #fig = sns.barplot(x = "mape", y = "modelo", data = df_mape, ax = ax5, orient = "h", color = 'grey')
    #ax5.set_xlabel("Valor de la métrica")
    #ax5.set_ylabel(" ")
    #ax5.tick_params(labelbottom = False, bottom = False)
    #for pa in ax5.patches:
    #    ax5.annotate("%.4f" % pa.get_width(), xy = (pa.get_width(), pa.get_y() + pa.get_height()/2),
    #        xytext = (5, 0), textcoords = 'offset points', ha = "left", va = "center")

Se importan las librerías necesarias

In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
pd.options.display.max_columns = None
pd.options.display.max_rows = None

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import cross_val_score

# Se importan los datos a usar para entrenar y validar el modelo

<div style = "float:right"><a style="text-decoration:none" href = "#Modelo-de-predicción:-Temperatura-ambiente">

In [ ]:
df_datos = pd.read_csv('./data/Modelo_2/Datos_modelo_2.csv', sep=',')

df_datos.head()

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/dsc/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3418, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-10-e4bc47267508>", line 1, in <module>
    df_datos = pd.read_csv('./data/Modelo_2/Datos_modelo_2.csv', sep=',')
  File "/home/dsc/anaconda3/lib/python3.8/site-packages/pandas/io/parsers.py", line 686, in read_csv
    return _read(filepath_or_buffer, kwds)
  File "/home/dsc/anaconda3/lib/python3.8/site-packages/pandas/io/parsers.py", line 458, in _read
    data = parser.read(nrows)
  File "/home/dsc/anaconda3/lib/python3.8/site-packages/pandas/io/parsers.py", line 1196, in read
    ret = self._engine.read(nrows)
  File "/home/dsc/anaconda3/lib/python3.8/site-packages/pandas/io/parsers.py", line 2155, in read
    data = self._reader.read(nrows)
  File "pandas/_libs/parsers.pyx", line 847, in pandas._libs.parsers.TextReader.read
  File "pandas/_libs/parsers.pyx", li

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/dsc/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3418, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-10-e4bc47267508>", line 1, in <module>
    df_datos = pd.read_csv('./data/Modelo_2/Datos_modelo_2.csv', sep=',')
  File "/home/dsc/anaconda3/lib/python3.8/site-packages/pandas/io/parsers.py", line 686, in read_csv
    return _read(filepath_or_buffer, kwds)
  File "/home/dsc/anaconda3/lib/python3.8/site-packages/pandas/io/parsers.py", line 458, in _read
    data = parser.read(nrows)
  File "/home/dsc/anaconda3/lib/python3.8/site-packages/pandas/io/parsers.py", line 1196, in read
    ret = self._engine.read(nrows)
  File "/home/dsc/anaconda3/lib/python3.8/site-packages/pandas/io/parsers.py", line 2155, in read
    data = self._reader.read(nrows)
  File "pandas/_libs/parsers.pyx", line 847, in pandas._libs.parsers.TextReader.read
  File "pandas/_libs/parsers.pyx", li

In [ ]:
#Selecciono las columnas de interés
df_datos = df_datos[["hora"] + list(df_datos.columns)[3:]]
df_datos.head()

ERROR! Session/line number was not unique in database. History logging moved to new session 1265


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/dsc/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3418, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-10-bff733bc63d0>", line 2, in <module>
    df_datos = df_datos[["hora"] + list(df_datos.columns)[3:]]
NameError: name 'df_datos' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/dsc/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 2045, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/dsc/anaconda3/lib/python3.8/site-packages/IPython/core/ultratb.py", line 1170, in get_records
    return _fixed_getinnerframes(etb, number_of_lines_of_context, tb_offset)
  File

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/dsc/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3418, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-10-bff733bc63d0>", line 2, in <module>
    df_datos = df_datos[["hora"] + list(df_datos.columns)[3:]]
NameError: name 'df_datos' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/dsc/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 2045, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/dsc/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3338, in run_ast_nodes
    if (await self.run_code(code, result,  async_=asy)):
  File "/home/d

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/dsc/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3418, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-10-bff733bc63d0>", line 2, in <module>
    df_datos = df_datos[["hora"] + list(df_datos.columns)[3:]]
NameError: name 'df_datos' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/dsc/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 2045, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/dsc/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3338, in run_ast_nodes
    if (await self.run_code(code, result,  async_=asy)):
  File "/home/d

Se determina la columna objetivo, las columnas útilies como entradas del modelo y las columnas mínimas para el modelo base

In [10]:
df_datos.shape

ERROR! Session/line number was not unique in database. History logging moved to new session 1266


NameError: name 'df_datos' is not defined

In [ ]:
columnas_utiles = list(df_datos.columns[:-2])
columnas_objetivo = ['temp_objetivo']
columnas_minimas = ['hora'] + list(columnas_objetivo)

### Separo las variables en x e y

En función de si son columna dato u objetivo

In [ ]:
df = df_datos

x = df[columnas_utiles]
y = df[columnas_objetivo]

display(x.head())
display(y.head())

# Se dividen los datos en train y test

<div style = "float:right"><a style="text-decoration:none" href = "#Modelo-de-predicción:-Temperatura-ambiente">

In [ ]:
x_train, x_test, y_train, y_test = dividir_train_test(x,y, 0.7)

# Estandarización
<div style = "float:right"><a style="text-decoration:none" href = "#Modelo-de-predicción:-Radiación-solar">

In [ ]:
x_train.head()

In [ ]:
x_test.head()

Se aplica la estandarización ``StandardScaler`` a los datos, eliminando la media y escalando los datos de forma que su varianza sea igual a 1

Conjunto de train

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
columnas = x_train.columns
x_train_normal = scaler.fit_transform(x_train[columnas])
x_train_normal = pd.DataFrame(x_train_normal, columns = columnas)

print(x_train_normal.head())

Se archiva el Scaler

In [ ]:
import pickle as pk

with open(directorio + 'data/Modelo_2/scaler_temp.pkl', 'wb') as file:
            pk.dump(scaler, file)

Conjunto de test

In [ ]:
columnas = x_test.columns
x_test_normal = scaler.transform(x_test[columnas])
x_test_normal = pd.DataFrame(x_test_normal, columns = columnas)

print(x_test_normal.head())

In [ ]:
x_train_normal.shape

In [ ]:
x_test_normal.shape

# PCA
<div style = "float:right"><a style="text-decoration:none" href = "#Modelo-de-predicción:-Temperatura-ambiente">

Se aplica PCA al dataset. El análisis de componentes principales (PCA) es una técnica utilizada para describir un conjunto de datos en términos de nuevas variables («componentes») no correlacionadas. El PCA busca la proyección según la cual los datos queden mejor representados en términos de mínimos cuadrados. Esta convierte un conjunto de observaciones de variables posiblemente correlacionadas en un conjunto de valores de variables sin correlación lineal llamadas componentes principales. 

Conjunto de train

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components = "mle")
principalComponents = pca.fit_transform(x_train_normal)
x_train_pca = pd.DataFrame(data = principalComponents)

En este caso, el número de componentes será:

In [ ]:
pca.n_components_

Se representa la varianza explicada acumulada en función del número de componentes, para tratar de buscar el número para el que la curva satura.

In [ ]:
plt.plot(np.cumsum(pca.explained_variance_ratio_))
plt.xlabel('number of components')
plt.ylabel('cumulative explained variance');

Se observa que la curva satura con 62 componentes, que serán las que se utilizen

In [ ]:
pca = PCA(n_components = 62)
principalComponents = pca.fit_transform(x_train_normal)
x_train_pca = pd.DataFrame(data = principalComponents)

In [ ]:
plt.plot(np.cumsum(pca.explained_variance_ratio_))
plt.xlabel('number of components')
plt.ylabel('cumulative explained variance');

Se archiva el PCA

In [ ]:
import pickle as pk

with open(directorio + 'data/Modelo_2/pca_temp.pkl', 'wb') as file:
            pk.dump(pca, file)

Conjunto de test

In [ ]:
principalComponents = pca.transform(x_test_normal)
x_test_pca = pd.DataFrame(data = principalComponents)

In [ ]:
x_train_pca.head()

In [ ]:
x_train_pca.shape

In [ ]:
x_test_pca.head()

In [ ]:
x_test_pca.shape

### Sanity check

Se comprueba si alguna de las componentes no sigue una distribución normal

In [ ]:
for i in range(3, len(x_train_pca.columns)):
    sns.set()
    cols = list(x_train_pca.columns)[i]
    sns.displot(x_train_pca[cols])
    plt.show();

Se comprueba que ninguna de las variables requiere del uso de transformaciones Yeo-Johnson o Box-Cox.

Se comprueba que las diferentes componentes no tiene relación entre si

In [ ]:
corrmat = x_train_pca[x_train_pca.columns[5:]].corr()
f, ax = plt.subplots(figsize = (12, 9))
sns.heatmap(corrmat, vmax = 0.8, square = True);

# Modelo base

<div style = "float:right"><a style="text-decoration:none" href = "#Modelo-de-predicción:-Temperatura-ambiente">

Se genera un primer modelo como base para las comparaciones. Este sencillamente obtiene el valor medio por hora de la función objetivo, asignando el valor predicción en función de la hora de los datos de entrada.

In [ ]:
# Datos a usar
datos_media = df[columnas_minimas]

# Se obtienen las medias por hora
datos_media = datos_media.groupby(columnas_minimas[:-1]).mean()
datos_media

In [ ]:
def modelo_medias_hora(datos, x_test):
    datos = pd.DataFrame(datos).reset_index()
    
    # Se seleccionan las columnas de interés
    x_test = x_test[columnas_minimas[:-1]]
    
    # Se asigna el valor medio por hora
    predicciones = []
    [predicciones.append([datos[datos[columnas_minimas[:-1][0]] == fila[columnas_minimas[:-1][0]]][[columnas_objetivo[0]]][columnas_objetivo[0]].iloc[0]][0]) for index,fila in x_test.iterrows()]

    return predicciones

In [ ]:
pred_media = modelo_medias_hora(datos_media, x_test)
pred_media[:1000]

Se comparan visualmente algunos valores ejemplo de predicción

In [ ]:
y_test_ = list(y_test.iloc[:,0])
graf_compara('Modelo medias horarias', y_test_[:100], pred_media[:100])

Se comparan visualmente los valores de predicción y la tendencia general de los valores objetivo

In [ ]:
sns.regplot(x = y_test, y = pred_media,  scatter_kws = {"color": "green"}, line_kws = {"color": "red"}).set_title('Modelo medias horarias');

Se obtienen las métricas de los resultados del modelo

In [ ]:
metricas_media = metricas('Modelo de medias horarias', y_test, pred_media)

In [ ]:
%store metricas_media

In [ ]:
%store -r metricas_media

In [ ]:
x_train = x_train_pca
x_test = x_test_pca

# Regresión lineal

<div style = "float:right"><a style="text-decoration:none" href = "#Modelo-de-predicción:-Temperatura-ambiente">

Se utiliza para estimar valores reales en función de variables continuas. Aquí, se establece la relación entre las variables independientes y dependientes mediante una recta. Esta línea de ajuste se conoce como línea de regresión y está representada por una ecuación lineal: ``Y = a * X + b``.

In [ ]:
from sklearn.linear_model import LinearRegression

# Creo el modelo y lo entreno
regre = LinearRegression()
regre.fit(x_train, y_train)

# Obtengo la predicción
pred_reg_lineal_ = regre.predict(x_test)

In [ ]:
pred_reg_lineal = []
[pred_reg_lineal.append(y[0]) for y in pred_reg_lineal_.tolist()]
pred_reg_lineal[:1000]

Se comparan visualmente algunos valores ejemplo de predicción

In [ ]:
graf_compara('Modelo de regresión lineal', list(y_test[columnas_objetivo[0]])[:100], pred_reg_lineal[:100])

Se comparan visualmente los valores de predicción y la tendencia general de los valores objetivo

In [ ]:
sns.regplot(x = y_test, y = pred_reg_lineal,  scatter_kws = {"color": "green"}, line_kws = {"color": "red"}).set_title('Modelo regresión lineal');

Se obtienen las métricas de los resultados del modelo

In [ ]:
metricas_regresion_lineal = metricas('Modelo de regresión lineal', y_test, pred_reg_lineal)

In [ ]:
compracion_metricas([metricas_media,
                     metricas_regresion_lineal])

In [ ]:
%store metricas_regresion_lineal

In [ ]:
%store -r metricas_regresion_lineal

# k-Nearest Neighbors

<div style = "float:right"><a style="text-decoration:none" href = "#Modelo-de-predicción:-Temperatura-ambiente">

Se puede utilizar tanto para problemas de clasificación como de regresión. kNN es un algoritmo simple que almacena todos los casos disponibles y clasifica los casos nuevos por mayoría de votos de sus k vecinos. El caso que se asigna a la clase es el más común entre sus K vecinos más cercanos medidos por una función de distancia.

In [ ]:
from sklearn.neighbors import KNeighborsRegressor

# Para un rango de número de puntos vecinos, se representa gráficamente la precisión del modelo
k_range = range(1, (len(x_train) if 5 > len(x_train) else 5))
scores = []
for k in k_range:
    knn = KNeighborsRegressor(n_neighbors = k, weights = 'uniform')
    knn.fit(x_train, y_train)
    scores.append(knn.score(x_test, y_test))
plt.figure();
plt.xlabel('k')
plt.ylabel('accuracy')
plt.scatter(k_range, scores);
k_ideal = []
[k_ideal.append(list(k_range)[i]) for i in range(0, len(scores)) if scores[i] == np.max(scores)]

In [ ]:
from sklearn.neighbors import KNeighborsRegressor

# Se crea y entrena el modelo con el valor de k obtenido
knn = KNeighborsRegressor(n_neighbors = k_ideal[0], weights = 'uniform')

knn.fit(x_train, y_train)

# Se obtiene la predicción
pred_knn_ = knn.predict(x_test)

In [ ]:
pred_knn = []
[pred_knn.append(y[0]) for y in pred_knn_.tolist()]
pred_knn[:10000]

Se comparan visualmente algunos valores ejemplo de predicción

In [ ]:
graf_compara('Modelo de knn', list(y_test[columnas_objetivo[0]])[:100], pred_knn[:100])

Se comparan visualmente los valores de predicción y la tendencia general de los valores objetivo

In [ ]:
sns.regplot(x = y_test, y = pred_knn,  scatter_kws = {"color": "green"}, line_kws = {"color": "red"}).set_title('Modelo de knn');

Se obtienen las métricas de los resultados del modelo

In [ ]:
metricas_knn = metricas('Modelo de knn', y_test, pred_knn)

In [ ]:
compracion_metricas([metricas_media,
                    metricas_regresion_lineal,
                    metricas_knn])

In [ ]:
%store metricas_knn

In [ ]:
%store -r metricas_knn

# Árbol de decisión

<div style = "float:right"><a style="text-decoration:none" href = "#Modelo-de-predicción:-Temperatura-ambiente">

Es un tipo de algoritmo de aprendizaje supervisado que funciona tanto para variables dependientes categóricas como continuas. Es un modelo predictivo que divide el espacio de los predictores agrupando observaciones con valores similares para la variable respuesta o dependiente.

In [ ]:
from sklearn.tree import DecisionTreeRegressor

# Se crea y entrena el modelo
tree = DecisionTreeRegressor (max_depth = 20,
                              min_samples_leaf = 15)

tree.fit(x_train, y_train)

# Se obtiene la predicción
pred_tree_ = tree.predict(x_test)

In [ ]:
pred_tree = []
[pred_tree.append(y) for y in pred_tree_.tolist()]
pred_tree[:1000]

Se comparan visualmente algunos valores ejemplo de predicción

In [ ]:
graf_compara('Modelo de árbol de decisión', list(y_test[columnas_objetivo[0]])[:100], pred_tree[:100])

Se comparan visualmente los valores de predicción y la tendencia general de los valores objetivo

In [ ]:
sns.regplot(x = y_test, y = pred_tree,  scatter_kws = {"color": "green"}, line_kws = {"color": "red"}).set_title('Modelo árbol de decisión');

Se obtienen las métricas de los resultados del modelo

In [ ]:
metricas_tree = metricas('Modelo de árbol de decisión', y_test, pred_tree)

In [ ]:
compracion_metricas([metricas_media,
                    metricas_regresion_lineal,
                    metricas_knn,
                    metricas_tree])

In [ ]:
%store metricas_tree

In [ ]:
%store -r metricas_tree

# Regresión polinomial 

<div style = "float:right"><a style="text-decoration:none" href = "#Modelo-de-predicción:-Temperatura-ambiente">

**NO SE UTILIZA**: PROVOCA PROBLEMAS DE MEMORIA

Es un caso especial de la regresión lineal, extiende el modelo lineal al agregar predictores adicionales, obtenidos al elevar cada uno de los predictores originales a una potencia. Por ejemplo, una regresión cúbica utiliza tres variables, como predictores. Este enfoque proporciona una forma sencilla de proporcionar un ajuste no lineal a los datos.

In [ ]:
from sklearn.base import BaseEstimator
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures 

# Se genera el estimador

class PolynomialRegression(BaseEstimator):
    def __init__(self, deg = None):
        self.deg = deg
            
    def fit(self, x, y, deg = None):
        self.model = LinearRegression()
        poli = PolynomialFeatures(degree = self.deg)
        x_ = poli.fit_transform(x)
        self.model.fit(x_, y)
    
    def predict(self, x):
        poli = PolynomialFeatures(degree = self.deg)
        x_ = poli.fit_transform(x)
        return self.model.predict(x_)

    def coef_(self):
        return pol_reg.coef_
    
    def best_params_(self):
        return pol_reg.best_params_

In [ ]:
from sklearn.preprocessing import PolynomialFeatures 
from sklearn.linear_model import LinearRegression

# Se crea y entrena el modelo

#pol_reg = LinearRegression()
#poli = PolynomialFeatures(degree = 2)
#x_train_pol = poli.fit_transform(x_train)
#x_test_pol = poli.fit_transform(x_test)
#pol_reg.fit(x_train_pol, y_train)

#pred_pol_reg_ = pol_reg.predict(x_test_pol)

# Gradient boosting

<div style = "float:right"><a style="text-decoration:none" href = "#Modelo-de-predicción:-Temperatura-ambiente">

Gradient boosting o Potenciación del gradiente, es una técnica de aprendizaje automático utilizado para el análisis de la regresión y para problemas de clasificación, el cual produce un modelo predictivo en forma de un conjunto de modelos de predicción débiles, típicamente árboles de decisión. Construye el modelo de forma escalonada como lo hacen otros métodos de boosting, y los generaliza permitiendo la optimización arbitraria de una función de pérdida diferenciable.

En Gradient Boosting, el número de árboles es un hiperparámetro crítico en cuanto que, con forme se añaden árboles, se incrementa el riesgo de overfitting.

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor

# Se crea y entrena el modelo
gbm = GradientBoostingRegressor(
            n_estimators = 10,
            loss         = 'ls',
            max_features = 'auto',
            random_state = 123
         )

gbm.fit(x_train, y_train.values.ravel())

# Se obtiene la predicción
pred_gbm_ = gbm.predict(x_test)

In [ ]:
pred_gbm = []
[pred_gbm.append(y) for y in pred_gbm_.tolist()]
pred_gbm[:1000]

Se comparan visualmente algunos valores ejemplo de predicción

In [ ]:
graf_compara('Modelo de Gradient boosting', list(y_test[columnas_objetivo[0]])[:100], pred_gbm[:100])

Se comparan visualmente los valores de predicción y la tendencia general de los valores objetivo

In [ ]:
sns.regplot(x = y_test, y = pred_gbm,  scatter_kws = {"color": "green"}, line_kws = {"color": "red"}).set_title('Modelo Gradient boosting');

Se obtienen las métricas de los resultados del modelo

In [ ]:
metricas_gbm = metricas('Modelo de Gradient boosting', y_test, pred_gbm)

In [ ]:
compracion_metricas([metricas_media,
                    metricas_regresion_lineal,
                    metricas_knn,
                    metricas_tree,
                    metricas_gbm])

In [ ]:
%store metricas_gbm

In [ ]:
%store -r metricas_gbm

# Random forest

<div style = "float:right"><a style="text-decoration:none" href = "#Modelo-de-predicción:-Temperatura-ambiente">

Random forest (o random forests) es una combinación de árboles predictores tal que cada árbol depende de los valores de un vector aleatorio probado independientemente y con la misma distribución para cada uno de estos. Es una modificación sustancial de bagging que construye una larga colección de árboles no correlacionados y luego los promedia.

In [ ]:
from sklearn.ensemble import RandomForestRegressor

# Se crea y entrena el modelo
# n_jobs: -1 means using all processors
forest = RandomForestRegressor(max_depth = None,
                              min_samples_leaf = 1,
                              n_estimators = 100,
                              n_jobs = -1)
                            
forest.fit(x_train,y_train.values.ravel())

# Se obtiene la predicción
pred_forest_ = forest.predict(x_test)

In [ ]:
pred_forest = []
[pred_forest.append(y) for y in pred_forest_.tolist()]
pred_forest[:1000]

Se comparan visualmente algunos valores ejemplo de predicción

In [ ]:
graf_compara('Modelo de random forest', list(y_test[columnas_objetivo[0]])[:100], pred_forest[:100])

Se comparan visualmente los valores de predicción y la tendencia general de los valores objetivo

In [ ]:
sns.regplot(x = y_test, y = pred_forest,  scatter_kws = {"color": "green"}, line_kws = {"color": "red"}).set_title('Modelo random forest');

Se obtienen las métricas de los resultados del modelo

In [ ]:
metricas_forest = metricas('Modelo de random forest', y_test, pred_forest)

In [ ]:
compracion_metricas([metricas_media,
                    metricas_regresion_lineal,
                    metricas_knn,
                    metricas_tree,
                    metricas_gbm,
                    metricas_forest])

In [ ]:
%store metricas_forest

In [ ]:
%store -r metricas_forest

# Se comprueba que no haya sobreajuste

Una solución para saber si un modelo sufre de "overfitting" es un procedimiento llamado validación cruzada (CV). En el enfoque básico, llamado k-fold CV, el conjunto de entrenamiento se divide en k conjuntos más pequeños. Se sigue el siguiente procedimiento para cada uno de los k "pliegues":
- Un modelo se entrena usando ``k-1`` de los pliegues como datos de entrenamiento;

- El modelo resultante se valida en la parte restante de los datos (es decir, se utiliza como un conjunto de pruebas para calcular una medida de rendimiento como la precisión).

Se recomienda que el número de folds f se calcule como: ``m/f > n^2`` siendo m x n las dimensiones del conjunto de datos

In [ ]:
x.shape

In [ ]:
f = (x.shape[0]/(x.shape[1])**2)/5
f

In [ ]:
scalar_sobredimension = StandardScaler()
pca_sobredimension = PCA(n_components = 62)
model = knn

pipeline = Pipeline([('transformer', scalar_sobredimension), ('pca', pca_sobredimension), ('estimator', model)])

scores = cross_val_score(pipeline, x, y.values.ravel(), cv = int(f), scoring = 'r2')
scores_numerico = []
[scores_numerico.append(float(i)) for i in list(scores)]
print(scores_numerico)
print("El valor medio de R² es {}, con desviación estándar de {}".format(np.mean(scores_numerico), np.std(scores_numerico)))

El mejor modelo entrenado hasta el momento es el de k-Nearest Neighbors, sin embargo, se puede comprobar que se encuentra sobreajustado

# GridSearchCV

<div style = "float:right"><a style="text-decoration:none" href = "#Modelo-de-predicción:-Temperatura-ambiente">

GridSearchCV es una clase disponible en scikit-learn que permite evaluar y seleccionar de forma sistemática los parámetros de un modelo. Indicándole un modelo y los parámetros a probar, puede evaluar el rendimiento del primero en función de los segundos mediante validación cruzada. Esta función ayuda a recorrer hiperparámetros predefinidos y ajustar su estimador (modelo) en su conjunto de entrenamiento. 

Por ello, se va a aplicar a los modelos previamente testeados, con r² como métrica

# Regresión lineal con GridSearchCV

<div style = "float:right"><a style="text-decoration:none" href = "#Modelo-de-predicción:-Temperatura-ambiente">

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression

# Se crea y entrena el modelo
regre_grid = GridSearchCV(LinearRegression(),
                       param_grid = {'fit_intercept':[True,False], 'normalize':[True,False], 'copy_X':[True, False]},
                       cv = 2,
                       scoring = "r2")

regre_grid.fit(x_train,y_train)

# Se obtiene la predicción
pred_reg_grid_ = regre_grid.predict(x_test)

#Parámetros escogidos
regre_grid.best_params_

In [ ]:
pred_reg_grid = []
[pred_reg_grid.append(y[0]) for y in pred_reg_grid_.tolist()]
pred_reg_grid[:1000]

Se comparan visualmente algunos valores ejemplo de predicción

In [ ]:
graf_compara('Modelo de regresión lineal con GridSearchCV', list(y_test[columnas_objetivo[0]])[:100], pred_reg_grid[:100])

Se comparan visualmente los valores de predicción y la tendencia general de los valores objetivo

In [ ]:
sns.regplot(x = y_test, y = pred_reg_grid,  scatter_kws = {"color": "green"}, line_kws = {"color": "red"}).set_title('Modelo regresión lineal con GridSearchCV');

Se obtienen las métricas de los resultados del modelo

In [ ]:
metricas_reg_grid = metricas('Modelo de regresión lineal con GridSearchCV', y_test, pred_reg_grid)

In [ ]:
compracion_metricas([metricas_media,
                    metricas_regresion_lineal,
                    metricas_knn,
                    metricas_tree,
                    metricas_gbm,
                    metricas_forest,
                    metricas_reg_grid])

In [ ]:
%store metricas_reg_grid

In [ ]:
%store -r metricas_reg_grid

# k-Nearest Neighbors con GridSearchCV

<div style = "float:right"><a style="text-decoration:none" href = "#Modelo-de-predicción:-Temperatura-ambiente">

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsRegressor

# Se crea y entrena el modelo
knn_grid = GridSearchCV(KNeighborsRegressor(),
                       param_grid = {"n_neighbors":np.arange(1, 6),
                                     "leaf_size":np.arange(10, 30)},
                       cv = 2,
                       scoring = "r2")


knn_grid.fit(x_train, y_train)

# Se obtiene la predicción
pred_knn_grid_ = knn_grid.predict(x_test)

#Parámetros escogidos
knn_grid.best_params_

In [ ]:
pred_knn_grid = []
[pred_knn_grid.append(y[0]) for y in pred_knn_grid_.tolist()]
pred_knn_grid[:1000]

Se comparan visualmente algunos valores ejemplo de predicción

In [ ]:
graf_compara('Modelo de knn con GridSearchCV', list(y_test[columnas_objetivo[0]])[:100], pred_knn_grid[:100])

Se comparan visualmente los valores de predicción y la tendencia general de los valores objetivo

In [ ]:
sns.regplot(x = y_test, y = pred_knn_grid,  scatter_kws = {"color": "green"}, line_kws = {"color": "red"}).set_title('Modelo knn con GridSearchCV');

Se obtienen las métricas de los resultados del modelo

In [ ]:
metricas_knn_grid = metricas('Modelo de knn con GridSearchCV', y_test, pred_knn_grid)

In [ ]:
compracion_metricas([metricas_media,
                    metricas_regresion_lineal,
                    metricas_knn,
                    metricas_tree,
                    metricas_gbm,
                    metricas_forest,
                    metricas_reg_grid,
                    metricas_knn_grid])

In [ ]:
%store metricas_knn_grid

In [ ]:
%store -r metricas_knn_grid

# Árbol de decisión con GridSearchCV

<div style = "float:right"><a style="text-decoration:none" href = "#Modelo-de-predicción:-Temperatura-ambiente">

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeRegressor

# Se crea y entrena el modelo
tree_grid = GridSearchCV(DecisionTreeRegressor(),
                       param_grid = {"max_depth":np.arange(25,35),
                                     "min_samples_leaf":np.arange(2,10)},
                       cv = 2,
                       scoring = "r2")

tree_grid.fit(x_train,y_train)

# Se obtiene la predicción
pred_tree_grid_ = tree_grid.predict(x_test)

# Parámetros escogidos
tree_grid.best_params_

In [ ]:
pred_tree_grid = []
[pred_tree_grid.append(y) for y in pred_tree_grid_.tolist()]
pred_tree_grid[:1000]

Se comparan visualmente algunos valores ejemplo de predicción

In [ ]:
graf_compara('Modelo de árbol de decisión con GridSearchCV', list(y_test[columnas_objetivo[0]])[:100], pred_tree_grid[:100])

Se comparan visualmente los valores de predicción y la tendencia general de los valores objetivo

In [ ]:
sns.regplot(x = y_test, y = pred_tree_grid,  scatter_kws = {"color": "green"}, line_kws = {"color": "red"}).set_title('Modelo árbol de decisión con GridSearchCV');

Se obtienen las métricas de los resultados del modelo

In [ ]:
metricas_tree_grid = metricas('Modelo de árbol de decisión con GridSearchCV', y_test, pred_tree_grid)

In [ ]:
compracion_metricas([metricas_media,
                    metricas_regresion_lineal,
                    metricas_knn,
                    metricas_tree,
                    metricas_gbm,
                    metricas_forest,
                    metricas_reg_grid,
                    metricas_knn_grid,
                    metricas_tree_grid])

In [ ]:
%store metricas_tree_grid

In [ ]:
%store -r metricas_tree_grid

# Gradient boosting con GridSearchCV

<div style = "float:right"><a style="text-decoration:none" href = "#Modelo-de-predicción:-Temperatura-ambiente">

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor

# Se crea y entrena el modelo

gbm_grid =  GridSearchCV(GradientBoostingRegressor(),
                       param_grid = {'n_estimators':[100,500], 
                                'learning_rate': [0.1,0.05,0.02],
                                'max_depth':[4], 
                                'min_samples_leaf':[3], 
                                'max_features':[1.0] },
                       cv = 2,
                       scoring = "neg_mean_absolute_error")
                            
gbm_grid.fit(x_train,y_train.values.ravel())

# Se obtiene la predicción
pred_gbm_grid_ = gbm_grid.predict(x_test)

# Parámetros escogidos
gbm_grid.best_params_

In [ ]:
pred_gbm_grid = []
[pred_gbm_grid.append(y) for y in pred_gbm_grid_.tolist()]
pred_gbm_grid[:1000]

Se comparan visualmente algunos valores ejemplo de predicción

In [ ]:
graf_compara('Modelo de gradient boosting con GridSearchCV', list(y_test[columnas_objetivo[0]])[:100], pred_gbm_grid[:100])

Se comparan visualmente los valores de predicción y la tendencia general de los valores objetivo

In [ ]:
sns.regplot(x = y_test, y = pred_gbm_grid,  scatter_kws = {"color": "green"}, line_kws = {"color": "red"}).set_title('Modelo gradient boosting con GridSearchCV');

Se obtienen las métricas de los resultados del modelo

In [ ]:
metricas_gbm_grid = metricas('Modelo de gradient boosting con GridSearchCV', y_test, pred_gbm_grid)

In [ ]:
compracion_metricas([metricas_media,
                    metricas_regresion_lineal,
                    metricas_knn,
                    metricas_tree,
                    metricas_gbm,
                    metricas_forest,
                    metricas_reg_grid,
                    metricas_knn_grid,
                    metricas_tree_grid,
                    metricas_gbm_grid])

In [ ]:
%store metricas_gbm_grid

In [ ]:
%store -r metricas_gbm_grid

# Random forest con Regressor RandomizedSearchCV

<div style = "float:right"><a style="text-decoration:none" href = "#Modelo-de-predicción:-Temperatura-ambiente">

RandomizedSearchCV implementa un método de "ajuste" y "puntuación". Los parámetros del estimador utilizados para aplicar estos métodos se optimizan mediante una búsqueda con validación cruzada sobre la configuración de parámetros. A diferencia de GridSearchCV, no se prueban todos los valores de los parámetros, sino que se muestrea un número fijo de configuraciones de parámetros de las distribuciones especificadas. El número de ajustes de parámetros que se prueban viene dado por n_iter.

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV

# Se crea y entrena el modelo

forest_random = RandomizedSearchCV(RandomForestRegressor(),
                       param_distributions={"max_depth":np.arange(25,35),
                                  "min_samples_leaf":np.arange(2,10),
                                  "n_estimators": (10,50,100,500)},
                       cv = 2,
                       scoring = "r2",
                       n_iter = 5)
                            
forest_random.fit(x_train,y_train.values.ravel())

# Se obtiene la predicción
pred_forest_random_ = forest_random.predict(x_test)

# Parámetros escogidos
forest_random.best_params_

In [ ]:
pred_forest_random = []
[pred_forest_random.append(y) for y in pred_forest_random_.tolist()]
pred_forest_random[:1000]

Se comparan visualmente algunos valores ejemplo de predicción

In [ ]:
graf_compara('Modelo de random forest con RandomizedSearchCV', list(y_test[columnas_objetivo[0]])[:100], pred_forest_random[:100])

Se comparan visualmente los valores de predicción y la tendencia general de los valores objetivo

In [ ]:
sns.regplot(x = y_test, y = pred_forest_random,  scatter_kws = {"color": "green"}, line_kws = {"color": "red"}).set_title('Modelo random forest con RandomizedSearchCV');

Se obtienen las métricas de los resultados del modelo

In [ ]:
metricas_forest_random = metricas('Modelo de random forest con RandomizedSearchCV', y_test, pred_forest_random)

In [ ]:
compracion_metricas([metricas_media,
                    metricas_regresion_lineal,
                    metricas_knn,
                    metricas_tree,
                    metricas_gbm,
                    metricas_forest,
                    metricas_reg_grid,
                    metricas_knn_grid,
                    metricas_tree_grid,
                    metricas_gbm_grid,
                    metricas_forest_random])

In [ ]:
%store metricas_forest_random

In [ ]:
%store -r metricas_forest_random

# Tensorflow

<div style = "float:right"><a style="text-decoration:none" href = "#Modelo-de-predicción:-Temperatura-ambiente">

TensorFlow es el sistema de aprendizaje automático de segunda generación de Google Brain, liberado como software de código abierto en 9 de noviembre de 2015. TensorFlow es una plataforma de código abierto de extremo a extremo para el aprendizaje automático. Cuenta con un ecosistema integral y flexible de herramientas, bibliotecas y recursos de la comunidad que les permite a los investigadores innovar con el aprendizaje automático y, a los desarrolladores, compilar e implementar con facilidad aplicaciones con tecnología de aprendizaje automático.

In [ ]:
#! pip install --upgrade tensorflow

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Activation,Dropout
from tensorflow.keras.models import Model

# Se crea y entrena el modelo

input_layer = Input(shape = (x_train_pca.shape[1],))
dense_layer_1 = Dense(100, activation = 'relu')(input_layer)
dense_layer_2 = Dense(50, activation = 'relu')(dense_layer_1)
dense_layer_3 = Dense(25, activation = 'relu')(dense_layer_2)
output = Dense(1)(dense_layer_3)

tensor = Model(inputs = input_layer, outputs = output)
tensor.compile(loss = "mean_squared_error" , optimizer = "adam", metrics = ["mean_squared_error"])

tensor.fit(x_train, y_train.values.ravel(), batch_size = 2, epochs = 100, verbose = 1, validation_split = 0.2)

# Se obtiene la predicción
pred_tensor_ = tensor.predict(x_test)

In [ ]:
pred_tensor = []
[pred_tensor.append(y[0]) for y in pred_tensor_.tolist()]
pred_tensor[:1000]

Se comparan visualmente algunos valores ejemplo de predicción

In [ ]:
graf_compara('Modelo de TensorFlow', list(y_test[columnas_objetivo[0]])[:100], pred_tensor[:100])

Se comparan visualmente los valores de predicción y la tendencia general de los valores objetivo

In [ ]:
sns.regplot(x = y_test, y = pred_tensor,  scatter_kws = {"color": "green"}, line_kws = {"color": "red"}).set_title('Modelo tensor flow');

Se obtienen las métricas de los resultados del modelo

In [ ]:
metricas_pred_tensor = metricas('Modelo de tensor flow', y_test, pred_tensor)

In [ ]:
compracion_metricas([metricas_media,
                    metricas_regresion_lineal,
                    metricas_knn,
                    metricas_tree,
                    metricas_gbm,
                    metricas_forest,
                    metricas_reg_grid,
                    metricas_knn_grid,
                    metricas_tree_grid,
                    metricas_gbm_grid,
                    metricas_forest_random,
                    metricas_pred_tensor])

In [ ]:
%store metricas_pred_tensor

In [ ]:
%store -r metricas_pred_tensor

# Selección de modelo
<div style = "float:right"><a style="text-decoration:none" href = "#Modelo-de-predicción:-Temperatura-ambiente">

Se muestra una comparativa de los resultados de los diferentes modelos

In [ ]:
fig, ax = plt.subplots(4,3, figsize = (15,22))
fig.suptitle('Valores predichos frente a reales', fontsize = 20, fontweight = 'bold')
fig.tight_layout()
fig.subplots_adjust(top = 0.95, hspace = 0.15)

sns.regplot(x = y_test, y = pred_media, ax = ax[0,0],
            scatter_kws = {"color": "green"}, line_kws = {"color": "red"}).set_title('Modelo medias horarias');
sns.regplot(x = y_test, y = pred_reg_lineal, ax = ax[0,1],
            scatter_kws = {"color": "green"}, line_kws = {"color": "red"}).set_title('Modelo regresión lineal');
sns.regplot(x = y_test, y = pred_knn, ax = ax[0,2],
            scatter_kws = {"color": "green"}, line_kws = {"color": "red"}).set_title('Modelo de knn');
sns.regplot(x = y_test, y = pred_tree, ax = ax[1,0],
            scatter_kws = {"color": "green"}, line_kws = {"color": "red"}).set_title('Modelo árbol de decisión');
sns.regplot(x = y_test, y = pred_gbm, ax = ax[1,1],
            scatter_kws = {"color": "green"}, line_kws = {"color": "red"}).set_title('Modelo Gradient boosting');
sns.regplot(x = y_test, y = pred_forest, ax = ax[1,2],
            scatter_kws = {"color": "green"}, line_kws = {"color": "red"}).set_title('Modelo random forest');
sns.regplot(x = y_test, y = pred_reg_grid, ax = ax[2,0],
            scatter_kws = {"color": "green"}, line_kws = {"color": "red"}).set_title('Modelo regresión lineal con GridSearchCV');
sns.regplot(x = y_test, y = pred_knn_grid,  ax = ax[2,1],
            scatter_kws = {"color": "green"}, line_kws = {"color": "red"}).set_title('Modelo knn con GridSearchCV');
sns.regplot(x = y_test, y = pred_tree_grid, ax = ax[2,2],
            scatter_kws = {"color": "green"}, line_kws = {"color": "red"}).set_title('Modelo árbol de decisión con GridSearchCV');
sns.regplot(x = y_test, y = pred_gbm_grid, ax = ax[3,0],
            scatter_kws = {"color": "green"}, line_kws = {"color": "red"}).set_title('Modelo gradient boosting con GridSearchCV');
sns.regplot(x = y_test, y = pred_forest_random, ax = ax[3,1], 
            scatter_kws = {"color": "green"}, line_kws = {"color": "red"}).set_title('Modelo random forest con RandomizedSearchCV');
sns.regplot(x = y_test, y = pred_tensor, ax = ax[3,2], 
            scatter_kws = {"color": "green"}, line_kws = {"color": "red"}).set_title('Modelo tensorflow');

for fila in ax:
    for columna in fila:
        columna.set_xlabel("")
        columna.set_ylabel("")

La media de valor de radiación solar en plano horizontal por hora (W/m²) es:

In [ ]:
# Se obtiene la media de la variable objetivo

df[columnas_objetivo].mean()

El mae del método de k-Nearest Neighbors con GridSearchCV sobre el valor medio es:

In [ ]:
metricas_pred_knn[1]/df[columnas_objetivo].mean()[0]*100

El modelo seleccionado es el de k-Nearest Neighbors, que cuenta con mayor r² (90 %) que el resto de modelos y menor error medio (15 % sobre el valor medio).

# Se comprueba que no haya sobreajuste

<div style = "float:right"><a style="text-decoration:none" href = "#Modelo-de-predicción:-Temperatura-ambiente">

Pruebo el modelo con otros grupos de datos aleatorios, para comprobar si los resultados son similares

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsRegressor

resultados = []

for i in range(1,5):
    
    # Se obtienen los datos
    df_fin = df_datos.sample(frac = 0.6, random_state = i)
    
    # Se dividen los datos en columnas objetivo y entrada y se dividen en train y test
    x_fin = df_fin[columnas_utiles]
    y_fin = df_fin[columnas_objetivo]
    x_train_fin, x_test_fin, y_train_fin, y_test_fin = dividir_train_test(x_fin,y_fin, 0.8)
    
    # Estandarización
    scaler = StandardScaler()
    columnas = x_train_fin.columns
    x_train_fin_normal = scaler.fit_transform(x_train_fin[columnas])
    x_train_fin_normal = pd.DataFrame(x_train_fin_normal, columns = columnas)
    
    columnas = x_test_fin.columns
    x_test_fin_normal = scaler.transform(x_test_fin[columnas])
    x_test_fin_normal = pd.DataFrame(x_test_fin_normal, columns = columnas)
    
    # PCA
    pca = PCA(n_components = 62)
    principalComponents = pca.fit_transform(x_train_fin_normal)
    x_train_fin_pca = pd.DataFrame(data = principalComponents)
    
    principalComponents = pca.transform(x_test_fin_normal)
    x_test_fin_pca = pd.DataFrame(data = principalComponents)
    
    # Se crea y entrena el modelo
    knn_fin = KNeighborsRegressor(n_neighbors = 4, leaf_size = 10, weights = 'uniform')

    knn_fin.fit(x_train_fin,y_train_fin.values.ravel())
    
    # Se obtiene la predicción
    pred_knn_fin_ = knn_fin.predict(x_test_fin)
    pred_knn_fin = []
    [pred_knn_fin.append(y) for y in pred_knn_fin_.tolist()]
    
    # Se obtienen las métricas del modelo
    metricas_final = metricas('Modelo de knn', y_test_fin, pred_knn_fin)
    
    
    # Se guardan los resultados
    resultados.append(metricas_final[4])

print("El valor medio de R² es {}, con desviación estándar de {}".format(np.mean(resultados), np.std(resultados)))

Se concluye que el modelo es válido.

# Se guarda el modelo

<div style = "float:right"><a style="text-decoration:none" href = "#Modelo-de-predicción:-Temperatura-ambiente">

In [ ]:
directorio = '/home/dsc/git/TFM/'

In [ ]:
import pickle

# Se guarda el modelo
modelo = "modelo_2_temp.pkl"

with open(directorio + 'data/Modelo_2/' + modelo, 'wb') as file:
        pickle.dump(knn, file)    